In [3]:
import numpy as np
import matplotlib.pyplot as plt
from src.datareader import DataReader
from src.fastspd import FastSPD
import overpy
from collections import defaultdict
import time

In [13]:
%%time
data_reader = DataReader("VED_dataset/*.csv", files_limit=1)
data = data_reader.get_data()

CPU times: total: 1.59 s
Wall time: 1.65 s


In [19]:
data[0].shape

(2104, 3)

In [20]:
def corners_of_map(data):
    max_long, min_long, max_lat, min_lat = -10e10, 10e10, -10e10, 10e10
    for row in data:
        max_long, min_long = max(max(row[:,1]),max_long), min(min(row[:,1]),min_long)
        max_lat, min_lat = max(max(row[:,2]),max_lat), min(min(row[:,2]),min_lat)
    return max_long, min_long, max_lat, min_lat

area = corners_of_map(data)
print(area)

(42.3255680556, 42.2203163889, -83.6740211111, -83.8041355556)


In [21]:
def get_listings(area):
    '''
    takes area (x1,y1,x2,y2) and returns lists of POIs from categories:
    (shops, industries, offices, education)
    '''
    def get_shops(data):
        api = overpy.Overpass()
        x1, x2 = data[1], data[0]
        y1, y2 = data[3], data[2]

        api_call = api.query(f'''(node["shop"]({x1},{y1},{x2},{y2});
                    node["building"="retail"]({x1},{y1},{x2},{y2});
                    node["building"="supermarket"]({x1},{y1},{x2},{y2});
                    node["healthcare"="pharmacy"]({x1},{y1},{x2},{y2});
                            );out;''')
        return api_call
    
    def get_offices(data):
        api = overpy.Overpass()
        x1, x2 = data[1], data[0]
        y1, y2 = data[3], data[2]

        api_call = api.query(f'''(node["office"]({x1},{y1},{x2},{y2});
                    node["building"="office"]({x1},{y1},{x2},{y2});
                    node["amenity"="bank"]({x1},{y1},{x2},{y2});
                            );out;''')
        return api_call
    
    def get_industries(data):
        api = overpy.Overpass()
        x1, x2 = data[1], data[0]
        y1, y2 = data[3], data[2]

        api_call = api.query(f'''(node["building"="industrial"]({x1},{y1},{x2},{y2});
                    node["building"="warehouse"]({x1},{y1},{x2},{y2});
                            );out;''')
        return api_call
    
    def get_education(data):
        api = overpy.Overpass()
        x1, x2 = data[1], data[0]
        y1, y2 = data[3], data[2]

        api_call = api.query(f'''(node["amenity"="college"]({x1},{y1},{x2},{y2});
                    node["amenity"="school"]({x1},{y1},{x2},{y2});
                    node["amenity"="university"]({x1},{y1},{x2},{y2});
                    node["amenity"="music_school"]({x1},{y1},{x2},{y2});
                            );out;''')
        return api_call
    
    
    shops_list = get_shops(area).nodes
    time.sleep(5)
    offices_list = get_offices(area).nodes
    time.sleep(5)
    industries_list = get_industries(area).nodes
    time.sleep(5)
    edu_list = get_education(area).nodes
    return shops_list, industries_list, offices_list, edu_list


listings = get_listings(area)
shops_list, industries_list, offices_list, edu_list = listings

In [22]:
def pois_for_point(point, listings):
    '''
    Calculates number of POIs from given listings in close range of given point
    
    example:
    point=(x1,y1,x2,y2)
    listings=( list_of_shops, list_of_offices, ... )
    '''
    def area_around_point(point):
        t = 0.005
        lati,longi = point
        x1, x2 = round(longi-t,4), round(longi+t,4)
        y1, y2 = round(lati-t,4), round(lati+t,4)
        return x1,y1,x2,y2
    
    def count_in_list(listing, area):
        counter = 0
        long1,lat1,long2,lat2 = area
        for node in listing:
            if lat1 < node.lat <lat2 and long1 < node.lon < long2:
                counter+=1
        return counter
    
    aoi = area_around_point(point)
    POIs_count = np.zeros(len(listings),dtype=np.int8)
    for c, category in enumerate(listings):
        POIs_count[c] = count_in_list(category, aoi)
    
    return POIs_count


example_point = (42.3051058, -83.6929794)
res = pois_for_point(example_point, listings)
print(f"Number of shops: {res[0]}; number of industries buildings: {res[1]}; number of offices buildings: {res[2]}; number of education buildings: {res[2]}")

Number of shops: 5; number of industries buildings: 0; number of offices buildings: 2; number of education buildings: 2


In [23]:
fastspd = FastSPD()
compressed_data = list()
for r,row in enumerate(data):
    compressed_data.append(fastspd.compress(row))

In [24]:
len(compressed_data)

658

In [53]:
%%time
from tqdm import tqdm 

def pois_matrix(data, listings):
    save_point = 1000
    spid = 0
    
    poi_res = [[] for _ in range(len(data))] 
    for i, trajectory in tqdm(enumerate(data), total=len(data)):
        # save trajectories to file every save_point
        # if t % save_point == 0 and t != 0:
        #     np.savetxt(f"pois3/pois_{spid}.csv", matrix_of_pois[save_point*spid:save_point*(spid+1)].reshape((save_point,-1)).astype(int), delimiter=";")
        #     spid += 1
        for j, row in enumerate(trajectory):
            lat, lon = row[1], row[2]
            point = lat, lon
            poi_res[i].append(pois_for_point(point,listings))
        
    return poi_res

result = pois_matrix(compressed_data, listings)

100%|██████████| 658/658 [02:41<00:00,  4.08it/s]

CPU times: total: 2min 35s
Wall time: 2min 41s


In [56]:
len(result), len(result[0]), len(result[1])

(658, 124, 127)

In [ ]:
with open("poi_small.csv") as f:
    for t in result:
        for pois in t:
            f.write()

In [65]:
np.array(result[0]).shape

(124, 4)

In [66]:
result = np.array(list(map(lambda x: np.array(x), result)), dtype=object)

In [67]:
np.save('poi_small.npy', result, allow_pickle=True)